<a href="https://colab.research.google.com/github/hassanjkhan/SALT-DATA/blob/main/addingSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model.max_seq_length = 120

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 31.5 MB/s 
     |████████████████████████████████| 1.2 MB 28.5 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 34.6 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d25728de2adf9e9135ea3a33a88c26d31d7eb60eba815831ab1e40dc6f84e79d
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import re 
from google.colab import files
import io

uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['new-tweets.csv']))
data.head()

Saving new-tweets.csv to new-tweets.csv


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,tweet,sentiment
0,0,0,0,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,2
1,1,1,1,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,1
2,2,2,2,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,1
3,3,3,3,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,1
4,4,4,4,Fri Mar 23 00:40:36 +0000 2018,Copy successful traders automatically with Bit...,1


In [ ]:
data = data.replace(to_replace='https?:\/\/\S+', value='', regex=True) # remove https urls 
data = data.replace(to_replace="www\.[a-z]?\.?(com)+|[a-z]+\.(com)", value='', regex=True) # remove www urls 
data = data.replace(to_replace="\[video\]", value='', regex=True) # remove videos that got replaced with [video]
data = data.replace(to_replace='{link}', value='', regex=True) # remove links 
data = data.replace(to_replace='&[a-z]+;', value='', regex=True) # remove HTML embedded characters
data = data.replace(to_replace='@[^ ]+', value='', regex=True) # remove @usernames 

data = data.sample(frac=1)
tweets = []
labels = []

for index, row in data.iterrows():
    if type(row['tweet']) is not float:
      tweets.append(row['tweet'])
      labels.append(row['sentiment'])

print(tweets[:100])
print(labels[:100])

['Best ', 'The Bitcoin Reserve: Everything You Need To Know About Keeping It Safe  Simple With Bitcoin, R   #crypto #currency', 'RT  Official Bitcoin Cloud #Airdrop 1\n\n\n\nLimited to 10,000 Members\nTotal Supply # 20M\n4M Coins Will be Air…', 'RT  Cum play with me on my premium snap before my purple pussy story is deleted!😽😽😽\n\n🚨JOIN FOR ONLY $25 FOR LIFE🚨\n\n⬇️Click here⬇️…', 'RT  Our listing on   // #tvrevolution #crypto #cryptocurrencies #bitcoin #ethereum #blockchain #…', 'RT  #Airdrop Ethereum White Airdrop round#1. 1000 #EthereumWhite will be giving to everyone who complete the google form. Y…', 'The great BearWhale has transferred his XRP...  #ripple #hodl #cryptocoin $xvg $btc $eth… ', 'RT  #TOPSECRET the next big #Bitcoin rally will not be fueled by #FOMO. It will be panic buying as people wake up to the fac…', 'Binance CEO Refutes Financial Watchdog Warning Reports  #blockchain #bitcoin #crypto', 'RT  Snowden: NSA Monitors Bitcoin Users\n\n\n\n#Blockchain #AI #Cryptics #IC

In [ ]:
embeddings = model.encode(tweets)

In [ ]:
print(len(embeddings))

50858


In [ ]:
training_embeddings = embeddings[:41000]
testing_embeddings = embeddings[41001:]

training_labels = labels[:41000]
testing_labels = labels[41001:]

X = training_embeddings
Y = training_labels

print(len(X))
print(len(Y))

41000
41000


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300).fit(X, Y)
clf.predict(testing_embeddings)
clf.score(testing_embeddings, testing_labels)

0.937100537688952

In [ ]:
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['PerformSentiment.csv']))
data.head()

Saving PerformSentiment.csv to PerformSentiment.csv


,Unnamed: 0,date,text,date_days,date_year,date_month
0,0,2021-07-05 23:26:48,Why ‘Cardano is just teasing us’\n\n💡 https://...,2021-07-05,2021,7
1,1,2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroa...,2021-07-05,2021,7
2,2,2021-07-05 23:26:36,LUNALAND ($LLN) will burn coin supply until we...,2021-07-05,2021,7
3,3,2021-07-05 23:26:24,#Ilovebitcoin because people never assume that...,2021-07-05,2021,7
4,4,2021-07-05 23:26:12,$bsc $btc #BNB #Bitcoin #BSC #EarnBNB #BTC htt...,2021-07-05,2021,7


In [ ]:
data = data.replace(to_replace='https?:\/\/\S+', value='', regex=True) # remove https urls 
data = data.replace(to_replace="www\.[a-z]?\.?(com)+|[a-z]+\.(com)", value='', regex=True) # remove www urls 
data = data.replace(to_replace="\[video\]", value='', regex=True) # remove videos that got replaced with [video]
data = data.replace(to_replace='{link}', value='', regex=True) # remove links 
data = data.replace(to_replace='&[a-z]+;', value='', regex=True) # remove HTML embedded characters
data = data.replace(to_replace='@[^ ]+', value='', regex=True) # remove @usernames 
text = []
for index, row in data.iterrows():
    if type(row['text']) is not float:
      text.append(row['text'])
      

In [ ]:
textEmbeddings = model.encode(text)

In [ ]:
# clf = MLPClassifier(random_state=1, max_iter=300).fit(X, Y)
sentiment = clf.predict(testing_embeddings)
print(sentiment[:100])

[1 0 1 2 1 2 0 1 2 2 0 2 1 1 1 2 2 2 2 2 1 1 1 1 1 2 1 1 1 0 2 2 2 2 1 2 1
 1 1 1 0 1 2 1 1 0 2 2 2 2 1 1 2 2 0 1 1 1 1 2 0 2 1 2 2 1 0 2 1 1 2 1 1 2
 2 2 1 1 1 2 2 2 1 0 1 2 1 2 1 2 2 2 2 1 2 0 2 1 1 1]


In [ ]:
data['sentiment'] = sentiment
data.to_csv('CSVFiles/nameoffile.csv')
